<a href="https://colab.research.google.com/github/VisheshJain112/AI-in-Trading/blob/master/Momentum%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving eod-quotemedia.csv to eod-quotemedia.csv


In [ ]:
import io
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import math
import scipy.stats as stats

df = pd.read_csv(io.BytesIO(uploaded['eod-quotemedia.csv']), parse_dates=['date'], index_col=False)
close=df.reset_index().pivot(index = 'date', columns= 'ticker', values ='adj_close')
print("Loaded data")

def shift_returns(returns,n):
  return returns.shift(n)


def resampling_price(close):
  price_date= pd.date_range(start='2013-07-01', end='2017-06-30', freq = 'M')
  
  
  
  strd = '2013-07-01'


  date1 = datetime.strptime(strd, "%Y-%m-%d")
  
  price_date=price_date.insert(0,date1)
  
  
  close=close.reindex(price_date,method='ffill')
 
  
  return close


resample_price=resampling_price(close)


def log_returns(resample_price):
  
  result_log=pd.DataFrame()
  for i in resample_price:
      
        price=pd.DataFrame(resample_price[i])
        data_log=((np.log(price) - np.log(price.shift())))
        logr=pd.DataFrame(data_log)
        
        logr.dropna(subset=[i], inplace=True)
        
        result_log = pd.concat([result_log, logr], axis=1, sort=False)
  
        
       

  
  return result_log
monthly_close=resample_price

k= log_returns(resample_price)
lookahead_returns = shift_returns(k, -1)
prev_returns=shift_returns(k,1)
def get_top_n(k,n):
  ts=[]
  for index,row in k.iterrows():
    tp=[]
    top_n=row.nlargest(n).values
    
    for i in row:
      if(i in top_n):
        tp.append(1)
      else:
        tp.append(0)
    ts.append(tp)
  trsi=pd.DataFrame(ts,k.index,k.columns)
  return trsi



long_stocks=get_top_n(k,50).apply(pd.value_counts)
for index,row in long_stocks.iterrows():
  if(index==1):
    longed_stocks=row.nlargest(10).index

    
worst_stocks=get_top_n(-k,50).apply(pd.value_counts)
for index,row in worst_stocks.iterrows():
  if(index==1):
    shorted_stocks=row.nlargest(10).index
returns= log_returns(resample_price).apply(lambda x : (np.power(2.71828,x))-1)



df_long=get_top_n(prev_returns,50)
df_short=get_top_n(-1*prev_returns,50)

def portfolio_returns(df_long,df_short,lookahead_returns):
  return ((df_long - df_short)*lookahead_returns)/100
expected_portfolio_returns = portfolio_returns(df_long,df_short,lookahead_returns)
expected_portfolio_plot = expected_portfolio_returns.T.sum()
expected_portfolio_plot.plot(x='date', y='Value', kind='line')
plt.show()   

expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))
  
def analyze_alpha(expected_portfolio_returns_by_date):

  res = stats.ttest_1samp(expected_portfolio_returns_by_date, 0.0)


  pvalue=res[1]/2
  tvalue=res[0]
  return tvalue,pvalue
t_value, p_value = analyze_alpha(expected_portfolio_returns_by_date)
print("""
Alpha analysis:
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(t_value, p_value))




  


Loaded data
ticker             A       AAL       AAP  ...       ZBH      ZION       ZTS
2013-07-01       NaN       NaN       NaN  ...       NaN       NaN       NaN
2013-07-31  0.025817  0.141314  0.007300  ...  0.107119  0.009832 -0.024321
2013-08-31  0.041814 -0.180153 -0.029776  ... -0.054021 -0.056639 -0.022389
2013-09-30  0.096579  0.159792  0.032823  ...  0.040284 -0.019860  0.065396
2013-10-31 -0.009607  0.147346  0.181959  ...  0.062871  0.034940  0.019229
2013-11-30  0.053881  0.066471  0.018283  ...  0.044059  0.033812 -0.016240
2013-12-31  0.067696  0.072677  0.091973  ...  0.021431  0.021252  0.048255
2014-01-31  0.016647  0.284211  0.036635  ...  0.008335 -0.041225 -0.071656
2014-02-28 -0.021203  0.095987  0.103739  ... -0.001384  0.083073  0.021506
2014-03-31 -0.017900 -0.008976 -0.006294  ...  0.010228 -0.007076 -0.069407
2014-04-30 -0.031825 -0.042702 -0.042058  ...  0.023201 -0.068808  0.047008
2014-05-31  0.052274  0.135525  0.023467  ...  0.075104 -0.010059  0.014436


AttributeError: ignored